In [19]:
import sys
import os



# Go two levels up: from 'notebooks/task1/' to the project root
sys.path.append(os.path.abspath(os.path.join( '..')))

In [20]:
import pandas as pd

df = pd.read_csv(r'C:\Users\ABC\Desktop\10Acadamy\week 2\playstore-bank-reviews\data\reviews_sentiment_comparison.csv')


In [ ]:
import oracledb

conn = oracledb.connect(user='bank_reviews', password='aym', dsn='localhost/XEPDB1')

cursor = conn.cursor()


In [ ]:

cursor.execute("""
CREATE TABLE reviews (
    review_id NUMBER GENERATED BY DEFAULT AS IDENTITY,
    review VARCHAR2(1000),
    rating NUMBER,
    review_date DATE,
    bank_id NUMBER,
    source VARCHAR2(100),
    detected_language VARCHAR2(10),
    distilbert_sentiment VARCHAR2(20),
    FOREIGN KEY (bank_id) REFERENCES banks(bank_id)
)
""")

In [8]:
# Rename the 'date' column in your DataFrame to 'review_date'
df = df.rename(columns={'date': 'review_date'})


In [9]:
banks_data = [
    ("Bank of Abissinya", "BOA", "com.boa.boaMobileBanking", "Ethiopia"),
    ("Commercial Bank of Ethiopia", "CBE", "com.combanketh.mobilebanking", "Ethiopia"),
    ("Dashen Bank", "Dashen", "com.dashen.dashensuperapp", "Ethiopia")
]

banks_insert_sql = """
INSERT INTO banks (bank_name, abbreviation, app_code, country)
VALUES (:1, :2, :3, :4)
"""

cursor.executemany(banks_insert_sql, banks_data)
conn.commit()




In [11]:
cursor.execute("SELECT bank_id, abbreviation FROM banks")
bank_id_map = {abbr: bank_id for bank_id, abbr in cursor.fetchall()}
print(bank_id_map)  # (Optional: check it looks right)


{'BOA': 1, 'CBE': 2, 'Dashen': 3}


In [14]:
insert_sql = """
INSERT INTO reviews (
    review, rating, review_date, bank_id, source, detected_language, distilbert_sentiment
) VALUES (
    :1, :2, TO_DATE(:3, 'YYYY-MM-DD'), :4, :5, :6, :7
)
"""

rows_to_insert = []
for idx, row in df.iterrows():
    bank_abbr = row['bank']  # Make sure this column matches your abbreviation
    bank_id = bank_id_map.get(bank_abbr)
    if not bank_id:
        continue  # Skip if bank abbreviation not found in map
    rows_to_insert.append((
        row['review'],
        int(row['rating']),
        str(row['review_date'])[:10],  # Must match your DataFrame's column name
        bank_id,
        row.get('source', ''),
        row.get('detected_language', ''),
        row.get('distilbert_sentiment', '')
    ))

# Optional: check that all rows are correct length
for i, vals in enumerate(rows_to_insert):
    if len(vals) != 7:
        print(f"Row {i} has {len(vals)} values: {vals}")

print(f"Inserting {len(rows_to_insert)} reviews...")

cursor.executemany(insert_sql, rows_to_insert)
conn.commit()
print("Done!")


Inserting 2839 reviews...
Done!


In [15]:
cursor.execute("SELECT COUNT(*) FROM reviews")
review_count = cursor.fetchone()[0]
print("Total reviews in table:", review_count)


Total reviews in table: 2839


In [18]:
cursor.execute("SELECT COUNT(*) FROM  banks")
banks_count = cursor.fetchone()[0]
print("Total  banks in table:",  banks_count)

Total  banks in table: 3


In [17]:
cursor.execute("""
SELECT r.review_id, r.review, r.rating, r.review_date, r.bank_id, r.source, r.detected_language, r.distilbert_sentiment
FROM reviews r
FETCH FIRST 10 ROWS ONLY
""")
for row in cursor.fetchall():
    print(row)


(1, '"Why don’t your ATMs support account-to-account transfers like other countries( Kenya, Nigeria , South africa)"', 4, datetime.datetime(2025, 6, 6, 0, 0), 2, 'Google Play', 'en', 'NEGATIVE')
(2, 'what is this app problem???', 1, datetime.datetime(2025, 6, 5, 0, 0), 2, 'Google Play', 'en', 'NEGATIVE')
(3, 'the app is proactive and a good connections.', 5, datetime.datetime(2025, 6, 5, 0, 0), 2, 'Google Play', 'en', 'POSITIVE')
(4, 'I cannot send to cbebirr app. through this app.', 3, datetime.datetime(2025, 6, 5, 0, 0), 2, 'Google Play', 'en', 'NEGATIVE')
(5, 'not functional', 1, datetime.datetime(2025, 6, 5, 0, 0), 2, 'Google Play', 'en', 'NEGATIVE')
(6, "everytime you uninstall the app you have to reach them out physically. very oldy! if that's one of security layer, they'd check for fraud attempt via app source directly(source:- app store, play store ... etc) implicitly ! we are in 2025, physical presence for every app install is traditional(traditionally very rare).", 1, datetim

In [21]:
from src.sql_csv_saving import export_table_to_csv
data_folder = r"C:\Users\ABC\Desktop\10Acadamy\week 2\playstore-bank-reviews\data\sql_datas"
export_table_to_csv(cursor, "reviews", data_folder)
export_table_to_csv(cursor, "banks", data_folder)


Exported reviews to C:\Users\ABC\Desktop\10Acadamy\week 2\playstore-bank-reviews\data\sql_datas\reviews_dump.csv (2839 rows)
Exported banks to C:\Users\ABC\Desktop\10Acadamy\week 2\playstore-bank-reviews\data\sql_datas\banks_dump.csv (3 rows)
